### This is code that will read in cp output images, classify them using a random tree classifier and save the parsed image.

In [7]:
import os
from scipy import ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import pandas as pd
from scipy import ndimage
from skimage import measure
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [8]:
orig_img_dir = r'Z:\TAD\190718_3t3l1_time_course_r5\stad3-28'
segm_img_dir = r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5'
annotated_csv_dir = r'C:\Users\LegantLab\Documents\git\tad\Legant_lab\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\img_analysis\annotated_csvs_pythong'
filtered_segm_img_dir = r'Z:\TAD\200428_adipogenesis_timecourses_revisited\190718_3t3l1_time_course_r5\filtered_segm_imgs'
#ase_img_name = 'stad3-29-xy1c1'

In [9]:
'''Train the classifier'''

os.chdir(annotated_csv_dir)
df_train_orig = pd.read_csv('annotated_df_orig.csv')


keep_cats = [ 'area',
 'bbox_area',
 'centroid-0',
 'centroid-1',
 'convex_area',
 'eccentricity',
 'equivalent_diameter',
 'extent',
 'local_centroid-0',
 'local_centroid-1',
 'major_axis_length',
 'minor_axis_length',
 'perimeter',
 'solidity',
 'inertia_tensor_eigvals-0',
 'inertia_tensor_eigvals-1',
 'moments_hu-0',
 'moments_hu-1',
 'moments_hu-2',
 'moments_hu-3',
 'moments_hu-4',
 'moments_hu-5',
 'moments_hu-6',
 'in_object']


df_train = df_train_orig[keep_cats] # make a copy of original df. I am going to alter this a little bit in order to pull out train and test cats
df_train = df_train.dropna()

'''this is target category for the training the classifier'''
target_cat_name = 'in_object'

'''define train and target data'''
x_train = df_train.drop('in_object', axis = 1)
y_train = df_train[target_cat_name]

'''this is a feature scaling step. This will standardize all of the data in order to pull everything into the same range'''
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)

'''random forest classifier'''
clf=RandomForestClassifier(n_estimators=100)

clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
'''make list of segmented images over which to iterate'''
os.chdir(segm_img_dir)

img_lst = []

for i in os.listdir():
    if '.tif' in i:
        img_lst.append(i)

In [21]:
'''iterate over images, apply classifier and generate new image'''


for segm_img_filename in img_lst:
    segm_img_orig = io.imread(segm_img_filename) #read in original image
    segm_img_orig = np.sum(segm_img_orig, axis = 2) #shift rbg into one channel
    labeled_img_orig = measure.label(segm_img_orig) #label the objects within the image 
    '''make a dataframe of object parameters'''
    df_test_orig = pd.DataFrame(measure.regionprops_table(labeled_img_orig, properties = ('label',  'area',  'bbox_area', 'centroid', 'convex_area', 
                                                       'eccentricity', 'equivalent_diameter',
                                                       'extent', 'local_centroid', 'major_axis_length', 
                                                                   'minor_axis_length', 'perimeter', 'solidity', 'inertia_tensor_eigvals',
                                                                            'moments_hu',)))
    '''generate a new data frame that contains only the predicted in objects'''
    df_test = df_test_orig[keep_cats[:-1]] #this pulls out categories for predicting
    x_test = sc_X.transform(df_test) #normalize all of the parameters 
    #df_test['in_object'] = clf.predict(x_test)
    df_test_orig['in_object'] = clf.predict(x_test) #add the prediction to the original df
    df_parsed = df_test_orig.loc[df_test_orig.in_object == 1] # make a dataframe that includes only the in objects
    
    '''generate and save image parsed image'''
    kept_labeled_objects = np.array(df_parsed.label) #make list of objects to keep
    kept_img_bool = np.isin(labeled_img_orig, kept_labeled_objects) #make a boolean image of things to keep
    labeled_img_parsed = np.multiply(kept_img_bool, labeled_img_orig) #this is the labeled image
    labeled_img_parsed = labeled_img_parsed.astype('uint16') #change the labeled image over to 16bit to be compatable with the other images
    '''save parsed image'''
    os.chdir(filtered_segm_img_dir)
    io.imsave(segm_img_filename[:-7] + 'segm_parsed.tiff', labeled_img_parsed)
    
    os.chdir(segm_img_dir)
    
    
    

C:\Users\LegantLab\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: stad3-26-xy1segm_parsed.tiff is a low contrast image
C:\Users\LegantLab\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: stad3-26-xy2segm_parsed.tiff is a low contrast image
C:\Users\LegantLab\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: stad3-26-xy3segm_parsed.tiff is a low contrast image
C:\Users\LegantLab\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: stad3-26-xy4segm_parsed.tiff is a low contrast image
C:\Users\LegantLab\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: stad3-26-xy5segm_parsed.tiff is a low contrast image
C:\Users\LegantLab\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: stad3-26-xy6segm_parsed.tiff is a low contrast image
C:\Users\LegantLab\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: stad3-26-xy7segm_parsed.tiff is a low contrast image
C:\Users\LegantLab\anaconda

In [19]:
segm_img_filename[:-7] + 'segm_parsed.tiff'

'stad3-28-xy7segm_parsed.tiff'

In [15]:
labeled_img_parsed.dtype

dtype('int64')